# Нграммы, коллокации и вероятности слов

В этом семинаре мы будем считать частотности слов и нграммов, находить устойчивые сочетания с помощью численных метрик, а также немного коснемся языкового моделирования.

Начнем с загрузки данных. Это кусочек корпуса новостей с ленты.ру (https://github.com/yutkin/Lenta.Ru-News-Dataset) В нем нет ничего особенного, просто его удобно использовать (ссылка лежит на гитхабе) и там достаточно много текстов (300гб) с классами.

In [1]:
# to download data
# !mkdir data
# !wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/lenta.txt.zip -P data
# !unzip -o data/lenta.txt.zip -d data/

In [1]:
# добавить разбиение не предложения чтобы не считать биграмы по границам предложений
corpus = open('data/lenta.txt').read()

In [2]:
corpus[:1000]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. \n«Русский инвалид», 16 сентября 1914 года.Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответств

Чтобы разделить текст одной строкой на токены, мы можем использовать простое регулярное выражение `\w+` или же воспользоваться готовым токенизатором из nltk - word_tokenize. Если посмотреть на код этого токенизатора, то можно увидеть, что он тоже основан на регулярных выражениях, но они посложнее и включают специфичные юзкейсы.

Сам токенизатор определяется тут - https://www.nltk.org/_modules/nltk/tokenize.html#word_tokenize  
Он ссылается на NLTKWordTokenizer отсюда - https://www.nltk.org/_modules/nltk/tokenize/destructive.html и как раз в нем определяются все регулярки

В этой функции также можно заметить опциональный токенизатор предложений. Нам он тоже понадобится, так как мы будем создавать нграммы и не хотелось бы, чтобы они собирались через границу предложений (последнее слово предыдущего предложения + первое слово следующего предложения). Но мы будем делать токенизацию на предложения отдельно, потому что в word_tokenize она используется, только чтобы улучшить саму токенизацию (на слова) и в итоге возвращается просто список токенов. 

In [4]:
# !pip install nltk

In [10]:
import nltk
nltk.download('punkt_tab')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/mnefedov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
sentences = sent_tokenize(corpus, language='russian')

In [12]:
# у word_tokenize есть еще параметры language и preserve_line
# но нам они не понадобятся, потому что они нужны только для sentence_tokenization
# сама токенизация в этой функции nltk не меняется в зависимости от языка
# но если вам нужен просто список токенов то имеет смысл указать язык и preserve_line=True
# чтобы немножко улучшить результаты

tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

In [13]:
# результат это список списков 
tokenized_sentences[:2]

[['Бои',
  'у',
  'Сопоцкина',
  'и',
  'Друскеник',
  'закончились',
  'отступлением',
  'германцев',
  '.'],
 ['Неприятель',
  ',',
  'приблизившись',
  'с',
  'севера',
  'к',
  'Осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью',
  '.']]

Нам не важны знаки пунктуации и регистр поэтому можем дофильтровать этот результат

In [14]:
import re

In [15]:
tokenized_sentences = [[token.lower() for token in sentence if not re.match('\W+', token)] 
                       for sentence in tokenized_sentences]

In [16]:
tokenized_sentences[:2]

[['бои',
  'у',
  'сопоцкина',
  'и',
  'друскеник',
  'закончились',
  'отступлением',
  'германцев'],
 ['неприятель',
  'приблизившись',
  'с',
  'севера',
  'к',
  'осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью']]

Теперь мы можем посчитать количество вхождений каждого слова

In [17]:
# Counter это структура данных в питоне, которая очень похожа на словарь
# но она заточена именно на подсчет 
# для этого можно использовать и простой словарь, но Counter просто удобнее
from collections import Counter

In [18]:
token_counts = Counter()
for sentence in tokenized_sentences:
    token_counts.update(sentence)

Если посмотреть на топ частотных слов, то там ожидаемо окажутся приставки и предлоги

In [19]:
token_counts.most_common(10)

[('в', 69941),
 ('и', 32908),
 ('на', 28127),
 ('по', 19233),
 ('что', 17030),
 ('с', 15737),
 ('не', 12689),
 ('из', 7708),
 ('как', 7370),
 ('о', 7117)]

Чтобы результат был чуть более интересным, можно удалить и корпуса стоп-слова.
Сам термин стоп-слово происходит из информационного поиска, первый раз его упомянул [Питер Лун](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) в 1959.  
Удаление таких слов позволяло сократить размер индекса и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали TFIDF (про него мы поговорим на одном из следующих семинаров).

В nltk есть базовые списки стоп-слов для разных языков. 

In [20]:
https://books.google.com/ngrams/graph?content=%D1%8D%D0%BA%D0%BE%D0%BD%D0%BE%D0%BC%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9+%D0%BA%D1%80%D0%B8%D0%B7%D0%B8%D1%81%2C%D1%80%D0%BE%D1%81%D1%82+%D1%86%D0%B5%D0%BD&year_start=1800&year_end=2022&corpus=ru&smoothing=3

In [21]:
# russian_stopwords

In [22]:

token_counts = Counter()
for sentence in tokenized_sentences:
    token_counts.update([token for token in sentence if token not in russian_stopwords])

In [20]:
token_counts.most_common(10)

[('россии', 5442),
 ('сообщает', 4591),
 ('также', 4087),
 ('года', 3602),
 ('новости', 3586),
 ('риа', 3517),
 ('это', 3273),
 ('время', 3168),
 ('словам', 2993),
 ('заявил', 2940)]

### Нграммы 

Теперь можно сделать тоже самое с сочетаниями из n токенов (нграммами). Напишем простую функцию, которая будет склеивать токены, которые стоят рядом

In [23]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [25]:
ngrammer('1234567')

['1 2', '2 3', '3 4', '4 5', '5 6', '6 7']

In [23]:
bigram_counts = Counter()
for sentence in tokenized_sentences:
    bigram_counts.update(ngrammer([token for token in sentence if token not in russian_stopwords]))

In [24]:
bigram_counts.most_common(10)

[('риа новости', 3501),
 ('сообщает риа', 1325),
 ('настоящее время', 769),
 ('миллионов долларов', 679),
 ('2000 года', 571),
 ('передает риа', 568),
 ('1999 года', 544),
 ('владимир путин', 519),
 ('федеральных сил', 492),
 ('эхо москвы', 488)]

In [25]:
trigram_counts = Counter()
for sentence in tokenized_sentences:
    # можно убрать фильтр стоп-слов
    trigram_counts.update(ngrammer([token for token in sentence], 3))

trigram_counts.most_common(20)

[('сообщает риа новости', 1324),
 ('со ссылкой на', 1242),
 ('по его словам', 889),
 ('в связи с', 834),
 ('в настоящее время', 752),
 ('о том что', 744),
 ('в том числе', 608),
 ('передает риа новости', 565),
 ('на северном кавказе', 470),
 ('риа новости в', 461),
 ('риа новости со', 428),
 ('новости со ссылкой', 408),
 ('в соответствии с', 367),
 ('в том что', 367),
 ('после того как', 355),
 ('сообщили риа новости', 332),
 ('до сих пор', 327),
 ('то же время', 325),
 ('как сообщает риа', 324),
 ('в то же', 324)]

N можно повышать и дальше, но выше 3-грамм на таком небольшом корпусе подниматься особого смысла нет, так как большинство будет иметь единичную частотность, а в топе будут совсем клишированные фразы. Но на больших данных, 4,5,6-граммы могут быть осмысленными. В целом такой простой подсчет частотностей по нграммам уже очень хороший способ что-то понять про данные и даже увидеть какие-то интересные зависимости. 

У гугла есть сервис, который показывает частотность нграмм по времени на огромном количестве данных - https://books.google.com/ngrams/ 

### Коллокации

В топе списков выше находятся нграммы, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Иногда это не то, что нужно. Нам могут быь интересуют случаи, когда слова в большинстве случаев встречаются вместе. 

Практический юзкейс для такой потребности - оптимизация размера словаря. Даже самые современные модели вроде gpt-4 работают с фиксированным словарем, от размера которого сильно зависит другой важный параметр - длина последовательности токенов, которую нужно пропускать через модель. Чем больше словарь, тем короче получается последовательность. Длина последовательности влияет на производительность сильнее, поэтому увеличивать словарь выгоднее (но до какого-то предела, потому что эмбединги/генерации отдельных слов все еще тоже нужны). Объединить токены, которые постоянно встречаются вместе (ведут себя как один токен) - очень эффективный способ сократить длину последовательности. (## и вообще это принцип по которому работает сжатие файлов - в них находятся повторяющиеся последовательности и их упоминания заменяются на какой-то более короткий символ по словарю ## )

Логично, что для поиска таких сочетаний (коллокаций) нам нужна какая-то формула, учитывающая частоты слов по отдельности и общую частоту, а также их какое-то отношений. Самый базовый способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности (униграмм). Такая метрика называется [Pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information).

Напишем функцию, которая будет высчитывать PMI из частот слова а и б и их общей частоты.

In [26]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        # идея тут простая, если слова встречают только вместе то 
        # bigram_count будет == word_count_a+word_count_b (а отношение == 0.5)
        # а чем меньше они встречаются вместе, тем меньше будет отношение 
        # в худшем случае будет 0 в знаменателе и 0 в результате
        score = bigram_count/ (word_count_a+word_count_b)
    
    except ZeroDivisionError:
        # на всякий случай
        return 0
    
    return score

# добавим стоп-слова сразу в нграммер для простоты
def ngrammer(tokens, n=2, stops=set()):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Сделаем функцию, которая будет делать счетчики для слов и биграммов.

In [27]:
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, 2, stops))
    
    return unigrams, bigrams

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.

In [28]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    
    for bigram in bigrams:
        word_a, word_b = bigram.split()
        score = scorer(unigrams[word_a], unigrams[word_b], 
                       bigrams[bigram])
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [29]:
unigrams, bigrams = collect_stats(tokenized_sentences, russian_stopwords)

In [30]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [31]:
bigram2score.most_common(15)

[('сопоцкина друскеник', 0.5),
 ('неприятель приблизившись', 0.5),
 ('саноку обстреливалась', 0.5),
 ('м.ю лермонтова', 0.5),
 ('австрийский аэроплан', 0.5),
 ('показывался аэроплан-птица', 0.5),
 ('das ist', 0.5),
 ('ist nesteroff', 0.5),
 ('песнь нестерове', 0.5),
 ('могучий унесся', 0.5),
 ('шумели лязгали', 0.5),
 ('зловеще гремели.и', 0.5),
 ('гремели.и пламенно', 0.5),
 ('жаждали битвы…величие', 0.5),
 ('равнине обманчиво-зыбкой.презрение', 0.5)]

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма.

In [32]:
def scorer(word_count_a, word_count_b, bigram_count, min_count=0):
    try:
        score = ((bigram_count - min_count) / ((word_count_a + word_count_b)))
    except ZeroDivisionError:
        return 0
    
    return score

# добавим параметр min_count
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=0):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    
    for bigram in bigrams:
        word_a, word_b = bigram.split()
        score = scorer(unigrams[word_a], unigrams[word_b], bigrams[bigram], min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [33]:
bigram2score = score_bigrams(unigrams, bigrams, scorer, min_count=20)

In [34]:
bigram2score.most_common(15)

[('риа новости', 0.4900746163592848),
 ('северном кавказе', 0.44553483807654565),
 ('associated press', 0.4345991561181435),
 ('new york', 0.4218009478672986),
 ('сих пор', 0.39092055485498106),
 ('взрывное устройство', 0.3665768194070081),
 ('таким образом', 0.3657187993680885),
 ('рао еэс', 0.33954451345755693),
 ('доменных имен', 0.31512605042016806),
 ('чрезвычайным ситуациям', 0.30935251798561153),
 ('налогам сборам', 0.30201342281879195),
 ('wall street', 0.3018867924528302),
 ('населенного пункта', 0.3013698630136986),
 ('объединенной группировки', 0.2993421052631579),
 ('возбуждено уголовное', 0.2983606557377049)]

В статье про Word2Vec для создания нграммов использовалась такая функция:

In [35]:
def scorer_w2v(word_count_a, word_count_b, bigram_count, len_vocab, min_count=0):

    try:
        score = ((bigram_count - min_count) * len_vocab) / (word_count_a * word_count_b)
    except ZeroDivisionError:
        return 0
    
    return score


# добавим параметр len_vocab
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=0):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    
    for bigram in bigrams:
        word_a, word_b = bigram.split()
        score = scorer(unigrams[word_a], unigrams[word_b], bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

Посмотрим, отличается ли она от нашей.

In [45]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_w2v, min_count=20)

In [46]:
bigram2score.most_common(15)

[('wall street', 1461.8575498575499),
 ('саудовской аравии', 1449.9000594883998),
 ('street journal', 1286.7391975308642),
 ('dow jones', 1263.8226600985222),
 ('подписных листов', 1223.637519872814),
 ('следственном изоляторе', 1204.1907114624505),
 ('чрезвычайным ситуациям', 1142.4925434962718),
 ('france presse', 1127.443359375),
 ('персидском заливе', 1122.9723905723906),
 ('полевые командиры', 1122.4325),
 ('полевых командиров', 1048.0228758169935),
 ('налогам сборам', 1045.7445652173913),
 ('следственный изолятор', 1002.171875),
 ('великой отечественной', 994.5958519092848),
 ('exit polls', 986.7538461538461)]

Во всех случаях выше мы считали нграммами только слова, которые встречаются друг за другом. Но в нграммы часто можно ещё что-то вставить. Например, "принять участие" может превратиться в "принять активное/непосредственное участие". 

Чтобы отловить такие случаи можно считать нграммами слова, которые встречаются внутри какого-то окна. И считать по ним все те же метрики.

Можно ещё посчитать стандартное отклонение расстояния между двумя словами. Если оно маленькое - слова обычно стоят на строгой позиции по отношению друг к другу.

In [52]:
from collections import defaultdict
import numpy as np

def get_window_stats(texts, window=8):
    
    bigrams = defaultdict(list)
    
    # проходим окном по текстам 
    # берем первое слово и считаем его целевым
    # проходим по остальным словам и их индексам
    # добавляем в словарь пары (целевое слов, текущее слово)
    # и добавляем индекс текущего в список этой пары
    # так мы получаем (слово_1,слово_2):[1,2,1,1,3,2]
    # порядок в этом случае учитывается - (слово_2, слово_1) - другая запись
    for text in texts:
        text = [token for token in text if token not in russian_stopwords]
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[2:]:
                bigrams[f'{target} {word}'].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        # выкидываем биграмы встретившиеся < 5 раз
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [53]:
bigrams_stds = get_window_stats(tokenized_sentences)

In [54]:
sorted(bigrams_stds, key=lambda x: bigrams_stds[x], reverse=False)

['международного фонда',
 'движения армии',
 'силой балла',
 'интерфаксу пресс-центре',
 'пресс-центре группировки',
 'тура выборов',
 'отечество россия',
 'агентству новости',
 'средствах информации',
 'the street',
 'the journal',
 'wall journal',
 'среду сентября',
 'единой сессии',
 'московской валютной',
 '9 1999',
 '4 утра',
 'борту находилось',
 'фсб генерал',
 'корреспонденту новости',
 'начала действий',
 'агентство новости',
 'самодельное устройство',
 'тех пока',
 '640 долларов',
 'международной организации',
 'видом жителей',
 'апреля года',
 'согласно законам',
 'нобелевской мира',
 'министр печати',
 'министр массовых',
 'министр коммуникаций',
 'телерадиовещания массовых',
 'телерадиовещания коммуникаций',
 'средств коммуникаций',
 'итар-тасс пресс-службу',
 'летом года',
 'corriere sera',
 'ночь воскресенье',
 'временно обязанности',
 'отдали голоса',
 'спустя минут',
 'блок вся',
 'мэр юрий',
 'министерство сша',
 'последние месяцев',
 'чеченский командир',
 'увд края'

По этой ссылке можно прочитать про другие метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

### Все готовое

Писать все это самому конечно не обязательно.

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы.

In [65]:
import gensim

In [118]:
# Мы можем использовать функцию выше напрямую (но нужно немножко поправить праметры)
# corpus_word_count не используется но он нужен по контракту

def scorer_w2v(worda_count, wordb_count, bigram_count, corpus_word_count, len_vocab=0,  min_count=0):

    try:
        score = ((bigram_count - min_count) * corpus_word_count) / (worda_count * wordb_count)
    except ZeroDivisionError:
        return 0
    
    return score

In [119]:
# собираем статистики
ph = gensim.models.Phrases(tokenized_sentences, 
                           min_count=1, 
                           threshold=1.,
                           scoring=scorer_w2v)

In [120]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про ворд2век и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [121]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[tokenized_sentences],  min_count=1, threshold=1., scoring=scorer_w2v)
p2 = gensim.models.phrases.Phraser(ph2)

In [258]:
tokenized_sentences[3]

['с',
 'раннего',
 'утра',
 '14',
 'сентября',
 'огонь',
 'достиг',
 'значительного',
 'напряжения']

In [122]:
p2[p[tokenized_sentences[3]]]

['с_раннего_утра',
 '14_сентября',
 'огонь',
 'достиг',
 'значительного',
 'напряжения']

Ну и наконец нграммы есть в нлтк. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [123]:
import nltk
from nltk.collocations import *

In [124]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [125]:
finder2 = BigramCollocationFinder.from_documents(tokenized_sentences, )

In [126]:
finder3 = TrigramCollocationFinder.from_documents(tokenized_sentences)

In [261]:
finder2.nbest(bigram_measures.pmi, 20, )

[('0,0277/', '0,0281'),
 ('0,06', 'миллиграм'),
 ('0,0698/', '0,07'),
 ('0,14', 'умарджбраилов'),
 ('0,1895/', '0,191'),
 ('0,191', '14,49'),
 ('0,238/', '0,2395'),
 ('0,70', 'процента.проведенное'),
 ('00:16', 'отделилась'),
 ('07.04.2000', '4233'),
 ('1,0510-', '1,0520'),
 ('1,22', 'рубля.стоимость'),
 ('1,85', 'милиона'),
 ('1,987', 'сек2'),
 ('1-ая', 'градская'),
 ('1-му', 'кадашевскому'),
 ('1-с', 'бухгалтерия'),
 ('1.4', 'срабатывает'),
 ('10-14', 'дней.россия'),
 ('10-ая', 'внеочередная')]

In [128]:
finder3.nbest(trigram_measures.lik, 20)

[('0,14', 'умарджбраилов', '0,08'),
 ('0,1895/', '0,191', '14,49'),
 ('1-ая', 'градская', '36-ая'),
 ('10-ая', 'внеочередная', 'сессиянационального'),
 ('10.417,06', 'пункта.intel', 'co.в'),
 ('103:123', '29:24', '26:32'),
 ('110kb', '248kb', '403kb'),
 ('12-летнего', 'володю', 'игошина'),
 ('12-этажном', 'блочном', 'одноподъездном'),
 ('137000', 'впарном', '73750'),
 ('14.17', 'ремонтным', 'бригадам'),
 ('19:25', '25:19', '25:16'),
 ('21:21', '42:35', '62:54'),
 ('22-летней', 'ольге', 'неврской'),
 ('24-летней', 'биатлонистки', 'мэри-бесс'),
 ('248kb', '403kb', '585kb'),
 ('26:32', '25:36', '23:31'),
 ('29:24', '26:32', '25:36'),
 ('33-летним', 'лоем', 'чоу'),
 ('34-летнему', 'юсуфу', 'крымшамхалову')]

### Вероятность слова

In [26]:
corpus_length = 0
token_counts = Counter()
for sentence in tokenized_sentences:
    token_counts.update(sentence)
    corpus_length += len(sentence)


probas = Counter({word:c/corpus_length for word, c in token_counts.items()})
probas.most_common(2000)

[('в', 0.04682045209139297),
 ('и', 0.02202953113943981),
 ('на', 0.018828996668257672),
 ('по', 0.01287510551856223),
 ('что', 0.011400356001721769),
 ('с', 0.010534785813217585),
 ('не', 0.008494369777207723),
 ('из', 0.005159949739358273),
 ('как', 0.004933683131690513),
 ('о', 0.004764317889856361),
 ('к', 0.004080162644186387),
 ('за', 0.004006525582519365),
 ('россии', 0.003643026269017608),
 ('для', 0.0033437920275161616),
 ('его', 0.003282874094682534),
 ('он', 0.0031402859661818446),
 ('от', 0.003086731739514919),
 ('сообщает', 0.0030733431828481876),
 ('а', 0.002905316796680709),
 ('также', 0.002735951554846557),
 ('будет', 0.0024494364421785058),
 ('года', 0.002411279055678321),
 ('новости', 0.002400568210344936),
 ('риа', 0.002354377689844713),
 ('до', 0.002341658561011318),
 ('это', 0.0021910372985105902),
 ('этом', 0.002189698442843917),
 ('время', 0.0021207473760102504),
 ('об', 0.0020471103143432277),
 ('словам', 0.0020035975051763506),
 ('заявил', 0.0019681178300095126

#### Работа с вероятностями (и просто маленькими числами) в питоне

У нас есть три основных способа посмотреть на конкретную вероятность:

1) в питоне по умолчанию используется запись маленьких и больших чисел через умножение на степень 10. Степень выводится сразу после символа e (если в начале степени стоит - , то это число меньше единицы). Чем больше/меньше степень, тем больше/меньше число. E используется просто как сокращение 10^, это не число e.

In [2]:
import numpy as np

In [7]:
print(5.1231/232312333221) # какое-то очень маленькое число

2.205263891489724e-11


Также можно самому задавать числа таким образом - это может сэкономить немного времени и сократить код. Так часто задают learning_rate в нейронных сетях 

In [27]:
1e1, 1e-1, 1e-4, 1e4, 1e-6

(10.0, 0.1, 0.0001, 10000.0)

2) для того, чтобы вывести привычную десятичную дробь, можно воспользоваться строковым методом .format с указанием количества знаков после запятой

In [32]:
print('{0:.30f}'.format(9.1231231241345123e-14))

0.000000000000091231231241345120


3) Вероятности слов обычно очень маленькие. С ними неудобно работать, а в расчетах с умножением вероятностей это вообще может привести к занулению (underflow). Поэтому часто используется логарифм вероятности (основание не имеет значения).  

In [9]:
print(np.log(9.1231231241345123e-14))

-30.025379108630506


#### Несколько фактов про логарифмы вероятностей

Вероятность это число от 0 до 1, поэтому её логарифм всегда будет от минус бесконечности до 0. 

In [34]:
np.log(0.0001), np.log(0.999), np.log(1)

(-9.210340371976182, -0.0010005003335835344, 0.0)

Логарифм от нуля равен бесконечности. Бесконечность сломает вычисления, в которых она окажется слагаемым

In [35]:
np.log(0) + np.log(1e100)

/Users/mnefedov/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

Есть специальная функция в numpy, которая добавляет к вероятности единицу прежде чем брать логарифм. Важное уточнение: нужно преобразовывать вероятности последовательно, каким-то одним способом - либо np.log либо np.log1p, нельзя для нуля делать np.log1p, а для всех остальных np.log!

In [273]:
np.log1p(0), np.log1p(1)

(0.0, 0.6931471805599453)

Для того, чтобы вернуться к обычной вероятности нужно взять экспоненту от логарифма вероятности. 

In [38]:
np.exp2(np.log2(0.1))

0.1

In [36]:
np.exp(np.log1p(0.1)) - 1

0.10000000000000009

Если вы использовали np.log1p, то вернутся поможет np.expm1 (она вычитает единицу после экспоненциирования)

In [59]:
np.expm1(np.log1p(0.1)), np.expm1(np.log1p(0.0))

(0.1, 0.0)

Умножение вероятностей тождественно сложению логарифмов вероятностей

In [39]:
(0.01*0.02)

0.0002

In [60]:
np.exp(np.log(0.01) + np.log(0.02)) 
# о том почему результат выглядит так странно можно почитать 
# вот тут - https://docs.python.org/3/tutorial/floatingpoint.html

0.0002000000000000002

## Вероятность текста

Перейти от вероятности отдельных слов к вероятностям целых текстов не так просто. Самый простой вариант, который мы можем попробовать - это рассматривать слова к тексте, как независимые события. И тогда общую вероятность (того, что из отдельных слов сложился данный текст) можно рассчитать просто умножив отдельные вероятности слов между собой. (Если бы мы хотели узнать вероятность сгенерировать какое-то любое слово из данного текста, то была бы сумма)

In [42]:
def normalize(text):
    sents = sent_tokenize(text)
    tokens = []
    for sent in sents:
        tokens += [token.lower() for token in word_tokenize(sent)]
    return tokens



def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(0.000000001))
    
    return np.exp(prob)

Рассчитаем вероятность встретить такой текст в нашем корпусе новостей (для таких маленьких чисел нужно смотреть на степень после e: чем больше степень, тем больше вероятность; но тут легко запутаться так как степень будет отрицательная и больше будет число, которое ближе к нулю (-5 больше -10 например)

In [46]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'
print(np.log(compute_joint_proba(phrase, probas)))
print('{0:.100f}'.format(np.exp(np.log(compute_joint_proba(phrase, probas)))))

-102.06520355013919
0.0000000000000000000000000000000000000000000047167765623735657467830797462035417875902989754196905332


In [44]:
phrase = 'сообщает РИА НОВОСТИ'
print(np.log(compute_joint_proba(phrase, probas)))
print('{0:.100f}'.format(np.exp(np.log(compute_joint_proba(phrase, probas)))))

-17.86851797762613
0.0000000173700569574918212807377649632700755688574645319022238254547119140625000000000000000000000000


Но естественно это слишком сильное упрощение.

In [47]:
# например такой текст будет более вероятен чем текст выше, хотя он (текст выше) взят из корпуса
phrase = "Рассчитаем вероятность встретить такой текст в каждом из корпусов"
# '{0:.50f}'.format(compute_joint_proba(phrase, probas))
print(np.log(compute_joint_proba(phrase, probas)))

-93.62260103027181


Слова в предложении - это не независимые события, выбор первого слова сильно влияет на вероятности выбрать второе, третье и так далее.

Такие события можно оценивать по формуле полной вероятности:

![](https://i.ibb.co/sC7CKzQ/image.png)

А если простыми словами, то для того, чтобы получить вероятность предложения, нужно перемножить `вероятность первого слова`, `вероятность второго слова, при условии первого`, `вероятность третьего при условии первого и второго`, `вероятность четвертого слова, при условии первого, второго и третьего` и так далее до вероятности последнего слова при условии всех предшешевствующих.

Условные вероятности для слов можно также вычислить по частотностям. Вероятность слова А при условии слова Б равна отношению количества раз, которое встретились слова А и Б вместе, к количеству раз, которое встретилось слово Б (или другими словами - сколько раз Б шло после А разделить на частоту А). Вероятность слова В при условии А и Б равна отношению количества раз, которое встретились слова А,Б и В вместе к количеству раз, которое встретились слова А и Б.
И так далее. 

Но тут появляется проблема. Для того, чтобы расчитать полную вероятность предложения нужно, чтобы такое предложение уже встретилось в корпусе хотя бы 1 раз. Очевидно, что даже огромный корпус всего написанного текста не включает в себя все возможные тексты (тем более маленький корпус). Поэтому один из множителей в произведении будет нулевым, а значит и все произведение станет нулевым.

![](https://i.ibb.co/Ctyq8DP/Screenshot-2024-09-18-at-14-17-43.png)


Для того, чтобы этого избежать можно поубавить строгости и предположить, что вероятность слова зависит только от предыдущего слова. Это предположение называется марковским (в честь математика Андрея Маркова). Такую модель еще можно назвать биграммной.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма. А частоты отдельных слов у нас уже есть. 

Вероятность первого слова можно по идее считать просто как вероятность униграмма, но можно сделать небольшое добавление в нашу модель - поставить в начала каждого предложения технический токен начала предложения, а вероятность первого слова рассчитывать как вероятность биграма старт-первое слово поделить на частоту старта. Дальше мы будем генерировать текст с помощью языковой модели и это поможет нам генерировать более красивые предложения.


Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [ ]:
def normalize(text):
    tokens += [token.lower() for token in word_tokenize(sent)]
    return tokens


In [185]:
lm_tokenized_sentences = [['<start>'] + sent + ['<end>'] for sent in tokenized_sentences]

In [186]:
lm_tokenized_sentences[0]

['<start>',
 'бои',
 'у',
 'сопоцкина',
 'и',
 'друскеник',
 'закончились',
 'отступлением',
 'германцев',
 '<end>']

In [187]:
unigrams = Counter()
bigrams = Counter()

for sentence in lm_tokenized_sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))

Чтобы посчитать условную вероятность мы можем поделить общее количество вхождений на количество вхождений первого слова. Обновим нашу функцию

In [86]:
def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

Теперь предложение из моего ноутбука не является более вероятным, чем кусок новостного текста

In [87]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

np.log(compute_joint_proba_markov_assumption(phrase, unigrams, bigrams))

-94.35322074721722

In [88]:
phrase = "Рассчитаем вероятность встретить такой текст в каждом из корпусов"
np.log(compute_joint_proba_markov_assumption(phrase, unigrams, bigrams))

-108.19778284410285

Даже Markov assumption не помогает в случае на картинке выше, потому что даже биграм "раму рано" не встречается в корпусе. Для этого можно использовать еще один трюк, когда мы при возможности считаем вероятность биграма, а если не получается, то используем отдельные вероятности. Мы также можем расширять Markov Assumption на триграммы/четырехграммы и т.д., тогда этот трюк будет поочереди пробовать сначала 4-грамм, потом 3-грамм, потом 2 грамм и потом отдельные вероятности. Строгость при этом конечно теряется, но это работает лучше, чем ничего. 

В статистических еще много разных усложнений и эвристик, которые помогают обойти проблемы с отсутствием сочетаний в корпусе или просто новыми словами. Про них можно почитать вот тут - https://web.stanford.edu/~jurafsky/slp3/3.pdf